# GNN Unlearning Notebook

## nvidia-smi

In [1]:
!nvidia-smi

Wed Nov 29 05:17:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Setup & Install Dependencies

In [ ]:
#@title Clone the inference repo
!git clone https://github.com/g-abhi/GNNDelete.git

%cd GNNDelete

In [ ]:
#@title pip install commands
# requirements.txt
!pip install -r requirements.txt

# torch
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

print(torch.__version__)

## Prepare Datasets

In [ ]:
#@title run prepare_dataset.py
!python prepare_dataset.py

## Train

### Node Classification

In [ ]:
# @title train base model for node classification

!python train_node.py -- checkpoint_dir "checkpoint_node" \
                      -- dataset "FacebookPagePage" \
                      -- unlearning_model "original"

### Edge/Link Prediction

In [ ]:
# @title train base model for edge/link prediction

!python train_gnn.py --lr 1e-3 \
                        --dataset "FacebookPagePage" \
                        --random_seed "42" \
                        --gnn "gcn"

# !python train_gnn.py --lr 1e-3 \
#                         --dataset "Cora" \
#                         --random_seed "100" \
#                         --gnn "gcn"

## Unlearn

### GNN Delete

In [ ]:
# @title unlearning_model="gnndelete"

!python delete_gnn.py --lr 1e-3 \
                        --epochs 500 \
                        --dataset "FacebookPagePage" \
                        --random_seed 42 \
                        --unlearning_model "gnndelete" \
                        --gnn "gcn" \
                        --df "out" \
                        --df_size 0.005 \
                        --checkpoint_dir "checkpoint_node" \
                        --out_dim 4

# !python delete_gnn.py --lr 1e-3 \
#                         --epochs 25 \
#                         --dataset "Cora" \
#                         --random_seed 100 \
#                         --unlearning_model "gnndelete" \
#                         --gnn "gcn" \
#                         --df "in" \
#                         --df_size 10

### Retrain

In [ ]:
# @title unlearning_model="retrain"

!python delete_gnn.py --lr 1e-3 \
                        --epochs 100 \
                        --dataset "FacebookPagePage" \
                        --random_seed 42 \
                        --unlearning_model "retrain" \
                        --gnn "gcn" \
                        --df "in" \
                        --df_size 10

# !python delete_gnn.py --lr 1e-3 \
#                         --epochs 25 \
#                         --dataset "Cora" \
#                         --random_seed 100 \
#                         --unlearning_model "retrain" \
#                         --gnn "gcn" \
#                         --df "in" \
#                         --df_size 10

### Graph Eraser

In [ ]:
# @title unlearning_model="graph_eraser"

!python delete_gnn.py --lr 1e-3 \
                        --epochs 500 \
                        --dataset "FacebookPagePage" \
                        --random_seed 42 \
                        --unlearning_model "graph_eraser" \
                        --gnn "gcn" \
                        --df "in" \
                        --df_size 10

## save zip files

In [ ]:
!zip -r /content/GNNDelete/checkpoint.zip /content/GNNDelete/checkpoint

In [ ]:
!zip -r /content/GNNDelete/data.zip /content/GNNDelete/data

In [ ]:
from google.colab import files
files.download("/content/GNNDelete/checkpoint.zip")
files.download("/content/GNNDelete/data.zip")